In [1]:
import finnhub
import pandas as pd
import numpy as np
import pyodbc
import time

finnhub_client = finnhub.Client(api_key="d5q4p8hr01qq2b6afv1gd5q4p8hr01qq2b6afv20")

for i in range(5) :
    try :
        finnhub_holidays = finnhub_client.market_holiday(exchange='US')
        break
    except Exception as e :
        print(f'Error : {e}. Retrying...{i}')
        time.sleep(5)

df = pd.DataFrame(finnhub_holidays['data'])
df.reset_index(drop=True, inplace=True)

df[['Start_Time', 'End_Time']] = df['tradingHour'].str.split('-', expand=True)
df['Status'] = np.where(df['tradingHour'].notna() & (df['tradingHour'] != ''), 'short day', 'closed')
df = df[['atDate', 'Status', 'Start_Time', 'End_Time', 'eventName']]

df.rename(columns = {
    "atDate" : "Date",
    "eventName" : "Description"},
    inplace = True)

df['Start_Time'] = df['Start_Time'].apply(lambda x: x if pd.notna(x) and x != '' else None)
df['End_Time'] = df['End_Time'].apply(lambda x: x if pd.notna(x) and x != '' else None)

sorted_df = df.sort_values(by='Date')

sorted_df['Date'] = pd.to_datetime(sorted_df['Date'])
sorted_df

,Date,Status,Start_Time,End_Time,Description
61,2023-01-02,closed,None,None,New Year's Day
60,2023-01-16,closed,None,None,"Birthday of Martin Luther King, Jr"
59,2023-02-20,closed,None,None,Washington's Birthday
58,2023-04-07,closed,None,None,Good Friday
57,2023-05-29,closed,None,None,Memorial Day
...,...,...,...,...,...
4,2027-07-05,closed,None,None,Independence Day
3,2027-09-06,closed,None,None,Labor Day
2,2027-11-25,closed,None,None,Thanksgiving Day
1,2027-11-26,short day,09:30,13:00,Thanksgiving Day


In [2]:
conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=Kiyoworld;"
    "DATABASE=USStocks;"
    "Trusted_Connection=yes;"
)
cursor = conn.cursor()

In [3]:
print("Loading Us Market Holiday Data into SQL Server...")

rows = list(sorted_df.itertuples(index=False, name=None))

try :
    insert_query = "INSERT INTO clean.us_market_holidays ([Date], [Status], Start_Time, End_Time, [Description]) VALUES (?, ?, ?, ?, ?)"
    cursor.fast_executemany = True
    cursor.executemany(insert_query, rows)
    conn.commit()

    print(f"Successfully inserted {len(sorted_df)} records.") 
    
except Exception as e :
    print(f"Error : {e}")

finally:
    cursor.close()
    conn.close()

Loading Us Market Holiday Data into SQL Server...
Successfully inserted 62 records.
